In [1]:
from __future__ import absolute_import
from __future__ import print_function

import csv
import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [3]:
module_path = '/home/wsliu/Codes/DLproj'
if module_path not in sys.path:
    sys.path.append(module_path)
if module_path+'/NRD' not in sys.path:
    sys.path.append(module_path+'/NRD')

In [6]:
data_path = '/nfs/turbo/umms-awaljee/wsliu/Data/MIMIC/raw/'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import os, sys, time
from importlib import reload
from keras.layers import Input, Embedding, Concatenate, Reshape, BatchNormalization, LSTM, CuDNNLSTM, CuDNNGRU, Lambda
from keras.models import Model
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.initializers import Constant
import keras.backend as K
from keras.layers import Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
#from tqdm import tqdm
from setsum_layer import SetSum, MaskedSum, MaskedDense, MaskedPooling
import tensorflow as tf

In [5]:
def dataframe_from_csv(path, header=0, index_col=0):
    return pd.read_csv(path, header=header, index_col=index_col)
e_map = {'ASIAN': 1,
         'BLACK': 2,
         'CARIBBEAN ISLAND': 2,
         'HISPANIC': 3,
         'SOUTH AMERICAN': 3,
         'WHITE': 4,
         'MIDDLE EASTERN': 4,
         'PORTUGUESE': 4,
         'AMERICAN INDIAN': 0,
         'NATIVE HAWAIIAN': 0,
         'UNABLE TO OBTAIN': 0,
         'PATIENT DECLINED TO ANSWER': 0,
         'UNKNOWN': 0,
         'OTHER': 0,
         '': 0}


def transform_ethnicity(ethnicity_series):
    global e_map

    def aggregate_ethnicity(ethnicity_str):
        return ethnicity_str.replace(' OR ', '/').split(' - ')[0].split('/')[0]

    ethnicity_series = ethnicity_series.apply(aggregate_ethnicity)
    return {'Ethnicity': ethnicity_series.fillna('').apply(lambda s: e_map[s] if s in e_map else e_map['OTHER'])}

In [57]:
admits = pd.read_csv(data_path+'ADMISSIONS.csv', header=0, index_col=0)
admits = admits[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME','ETHNICITY']]
admits.ADMITTIME = pd.to_datetime(admits.ADMITTIME)
admits.DISCHTIME = pd.to_datetime(admits.DISCHTIME)
admits.DEATHTIME = pd.to_datetime(admits.DEATHTIME)
admits['Eth']=transform_ethnicity(admits['ETHNICITY'])['Ethnicity']

admits.drop(columns=['ETHNICITY'],inplace=True)

patients =  pd.read_csv(data_path+'PATIENTS.csv', header=0, index_col=0)
patients = patients[['SUBJECT_ID','DOB','GENDER','DOD']]
patients.DOB = pd.to_datetime(patients.DOB)
patients.DOD = pd.to_datetime(patients.DOD)
patients.GENDER = patients.GENDER.astype('category')
patients.GENDER = patients['GENDER'].cat.codes
stays = pd.read_csv(data_path+'ICUSTAYS.csv',header=0,index_col=0)
stays = stays[['HADM_ID','LOS']]
#stays.INTIME = pd.to_datetime(stays.INTIME)
#stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
stays.drop_duplicates('HADM_ID',inplace=True)
admits = admits.merge(stays,how='inner',on='HADM_ID')
#admits.LOS[admits.LOS.isnull()] = (admits.DISCHTIME-admits.ADMITTIME).apply(lambda s: s/np.timedelta64(1,'D'))[admits.LOS.isnull()] 
admits = admits.merge(patients,how='inner',on='SUBJECT_ID')
admits['AGE'] = (admits.ADMITTIME-admits.DOB).apply(lambda s: s/np.timedelta64(1,'Y'))
admits.loc[admits.AGE < 0, 'AGE'] = 90
admits['MOT'] =  (admits.DOD-admits.ADMITTIME).apply(lambda s: s/np.timedelta64(1,'Y'))
mortality = (admits.DEATHTIME.isnull() | ((admits.ADMITTIME > admits.DEATHTIME) | (admits.DISCHTIME < admits.DEATHTIME)))
admits = admits[mortality]
codes = dataframe_from_csv(data_path+'D_ICD_DIAGNOSES.csv')
codes = codes[['ICD9_CODE']]
diagnoses = dataframe_from_csv(data_path+'DIAGNOSES_ICD.csv')
diagnoses = diagnoses.merge(codes, how='inner', left_on='ICD9_CODE', right_on='ICD9_CODE')
diagnoses = diagnoses[['HADM_ID','ICD9_CODE']]
diagnoses[ 'HADM_ID'] = diagnoses[ 'HADM_ID'].astype(int)
procedure_codes = dataframe_from_csv(data_path+'D_ICD_PROCEDURES.csv')
procedure_codes = procedure_codes[['ICD9_CODE']]
procedures = dataframe_from_csv(data_path+'PROCEDURES_ICD.csv')
procedures = procedures.merge(procedure_codes, how='inner', left_on='ICD9_CODE', right_on='ICD9_CODE')
procedures = procedures[['HADM_ID','ICD9_CODE']]
procedures['HADM_ID'] = procedures['HADM_ID'].astype(int)
diag_freq = diagnoses.groupby('ICD9_CODE',as_index=False).count()
proc_freq = procedures.groupby('ICD9_CODE',as_index=False).count()
procedures_original = procedures
diagnoses_original = diagnoses
admits['LOS1'] = (admits.DISCHTIME-admits.ADMITTIME).apply(lambda s: s/np.timedelta64(1,'D'))
admits.loc[admits.LOS.isnull(),'LOS'] = admits.loc[admits.LOS.isnull(),'LOS1']

In [68]:
admits.MOT.value_counts(dropna=False)

NaN          35744
 0.790428        4
 0.349625        4
 0.029290        4
 0.628892        4
 0.623416        3
 2.539951        3
 0.037504        3
 1.192900        3
 0.483782        3
 0.074302        3
 0.166185        3
 2.964326        3
 2.816479        3
 0.179875        3
 0.171661        3
 0.163447        3
 0.574133        3
 0.099705        2
 0.565920        2
 0.164554        2
 0.604250        2
-0.001936        2
 1.211296        2
 0.047006        2
 0.358961        2
 0.099143        2
 1.091512        2
 0.066392        2
 0.071577        2
             ...  
 2.725142        1
 0.115021        1
 1.150642        1
 0.433167        1
 0.057540        1
 1.278682        1
 0.225240        1
 0.498240        1
 1.498782        1
 0.293175        1
 0.115013        1
 0.198312        1
 6.809764        1
 0.509912        1
 0.416666        1
 0.653021        1
 0.044900        1
 0.240451        1
 1.422497        1
 8.277856        1
 0.225805        1
 0.137909   

In [29]:
diag_freq.head()

,ICD9_CODE,HADM_ID
0,0030,2
1,0031,1
2,0038,2
3,0039,1
4,0041,3


In [58]:
cutoff=40

In [59]:
diag_freq = diag_freq[~(diag_freq.HADM_ID<=cutoff)]
diagnoses_cut = diagnoses.merge(diag_freq[['ICD9_CODE']],how='right',on='ICD9_CODE')
proc_freq = proc_freq[~(proc_freq.HADM_ID<=cutoff)]
procedure_cut = procedures.merge(proc_freq[['ICD9_CODE']],how='right',on='ICD9_CODE')
diagnoses_count = diagnoses_original.groupby('HADM_ID',as_index=False).count()

In [49]:
diag_freq.shape, diagnoses_cut.shape, diagnoses.shape, diagnoses_count.shape

((1407, 2), (593863, 2), (634709, 2), (58925, 2))

In [53]:
diagnoses_count.ICD9_CODE.value_counts()

9     6088
8     5829
7     4686
6     3987
5     3652
4     3437
3     3025
10    2762
11    2714
12    2540
13    2319
14    2249
15    1967
16    1823
17    1600
18    1441
19    1248
2     1140
20    1016
21     896
22     717
23     563
24     483
25     407
1      399
26     349
27     277
28     265
30     185
29     185
31     128
32     112
33      86
39      79
36      69
35      60
34      59
38      43
37      40
Name: ICD9_CODE, dtype: int64

In [60]:
diagnoses_count = diagnoses_count.rename(str, columns={"ICD9_CODE":"COUNT"})
diagnoses_count = diagnoses_count[['HADM_ID','COUNT']]
diagnoses_count = diagnoses_count[diagnoses_count.COUNT<=30]
diagnoses_count = diagnoses_count[diagnoses_count.COUNT>0]

In [47]:
diagnoses_count.shape

(58249, 2)

In [61]:
procedures_count = procedures_original.groupby('HADM_ID',as_index=False).count()
procedures_count = procedures_count.rename(str, columns={"ICD9_CODE":"COUNT"})
procedures_count = procedures_count[['HADM_ID','COUNT']]
procedures_count = procedures_count[procedures_count.COUNT<=15]
procedures_count = procedures_count[procedures_count.COUNT>0]
diagnoses = diagnoses_cut.merge(diagnoses_count,'right',left_on='HADM_ID',right_on='HADM_ID')

In [62]:
diagnoses.head()

,HADM_ID,ICD9_CODE,COUNT
0,172335,40301,14
1,172335,486,14
2,172335,58281,14
3,172335,5855,14
4,172335,4254,14


In [63]:
diagnoses = diagnoses[diagnoses.COUNT>0]
diagnoses.ICD9_CODE = pd.Categorical(diagnoses.ICD9_CODE)

In [64]:
diagnoses.head()

,HADM_ID,ICD9_CODE,COUNT
0,172335,40301,14
1,172335,486,14
2,172335,58281,14
3,172335,5855,14
4,172335,4254,14


In [31]:
diagnoses.ICD9_CODE = (diagnoses.ICD9_CODE.cat.codes+1).astype(int)
procedures = procedure_cut.merge(procedures_count,'right',left_on='HADM_ID',right_on='HADM_ID')
procedures.ICD9_CODE = pd.Categorical(procedures.ICD9_CODE)
procedures.ICD9_CODE = (procedures.ICD9_CODE.cat.codes+1+max(diagnoses.ICD9_CODE)).astype(int)
n_disease=max(procedures.ICD9_CODE)+1
n_disease

1847

In [32]:
diagnoses_count.head()

,HADM_ID,COUNT
0,100001,16
1,100003,9
2,100006,9
3,100007,4
4,100009,18


In [7]:
diag_tempt = diagnoses.merge(diag_freq,how='left',on='ICD9_CODE')
diag_set = diag_tempt.HADM_ID_x[diag_tempt.HADM_ID_y<=cutoff]
diagnoses_cut = diagnoses[(~diagnoses['HADM_ID'].isin(diag_set))]
proc_tempt = procedures.merge(proc_freq,how='left',on='ICD9_CODE')
proc_set = proc_tempt.HADM_ID_x[proc_tempt.HADM_ID_y<=cutoff]
procedure_cut = procedures[~procedures['HADM_ID'].isin(proc_set)]
diagnoses_count = diagnoses_original.groupby('HADM_ID',as_index=False).count()
diagnoses_count = diagnoses_count.rename(str, columns={"ICD9_CODE":"COUNT"})
diagnoses_count = diagnoses_count[['HADM_ID','COUNT']]
diagnoses_count = diagnoses_count[diagnoses_count.COUNT<=30]
procedures_count = procedures_original.groupby('HADM_ID',as_index=False).count()
procedures_count = procedures_count.rename(str, columns={"ICD9_CODE":"COUNT"})
procedures_count = procedures_count[['HADM_ID','COUNT']]
procedures_count = procedures_count[procedures_count.COUNT<=15]
diagnoses = diagnoses_cut.merge(diagnoses_count,'left',left_on='HADM_ID',right_on='HADM_ID')
diagnoses = diagnoses[diagnoses.COUNT>0]
diagnoses.ICD9_CODE = pd.Categorical(diagnoses.ICD9_CODE)
diagnoses.ICD9_CODE = (diagnoses.ICD9_CODE.cat.codes+1).astype(int)
procedures = procedure_cut.merge(procedures_count,'left',left_on='HADM_ID',right_on='HADM_ID')
procedures.ICD9_CODE = pd.Categorical(procedures.ICD9_CODE)
procedures.ICD9_CODE = (procedures.ICD9_CODE.cat.codes+1+max(diagnoses.ICD9_CODE)).astype(int)
n_disease=max(procedures.ICD9_CODE)+1
n_disease

In [8]:
mortality1y = (admits.MOT <1) & admits.DOD.notnull()
#mortality = mortality | (admits.DEATHTIME.notnull() & ((admits.ADMITTIME <= admits.DEATHTIME) & (admits.DISCHTIME >= admits.DEATHTIME)))
#admits['MORTALITY'] = mortality.astype(int)
admits['MORTALITY_1year'] =mortality1y.astype(int)
training_df = admits[['HADM_ID','MORTALITY_1year','AGE','GENDER','LOS','Eth']]
diagnoses = diagnoses[['HADM_ID','ICD9_CODE']]
diagnoses_df = diagnoses.pivot_table(index='HADM_ID',columns=diagnoses.groupby('HADM_ID').cumcount(),values='ICD9_CODE',aggfunc=np.sum).add_prefix('DX')
procedures_df = procedures.pivot_table(index='HADM_ID',columns=procedures.groupby('HADM_ID').cumcount(),values='ICD9_CODE',aggfunc=np.sum).add_prefix('PR')
training_df = training_df.merge(diagnoses_df,how='inner',left_on='HADM_ID',right_on='HADM_ID')
training_df = training_df.merge(procedures_df,how = 'inner',left_on='HADM_ID',right_on='HADM_ID')
df_train = training_df
df_train.fillna(0,inplace=True)